### This notebook is to prepare data for analysis
#### Group Project on 'Public_Health' (Focus: Substance abuse in young adults)
#### Members: Jamuna, Eddie, Warren, Anna
#### Start Date: 05/15/2019

In [59]:
#Dependencies
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as ax
from scipy import stats
import seaborn as sns

In [60]:
# Explore input data from the csvs
adm_csvdata = pd.read_csv("admissions.csv")
dis_csvdata = pd.read_csv("discharges.csv")
print(f"Total number of records in admissions.csv is : %d" %len(adm_csvdata))
print(f"Total number of records in discharges.csv is : %d" %len(dis_csvdata))
#csvdata.columns

Total number of records in admissions.csv is : 1048575
Total number of records in discharges.csv is : 1048575


In [61]:
# Apply filters to select records of interest for analysis (rows filtered based on Primary substance - SUB1)
adm_data = adm_csvdata[adm_csvdata['SUB1'].isin([1,2,3,4,5,7,13,19]) & adm_csvdata['AGE'].isin([2,3,4]) ]
dis_data = dis_csvdata[dis_csvdata['SUB1'].isin([1,2,3,4,5,7,13,19]) & dis_csvdata['AGE'].isin([2,3,4])]
#adm_csvdata[adm_csvdata['SUB1'].isin[1,2,3,4,5,7,13,19]]
print(f"Total number of records filtered from admissions.csv is : %d" %len(adm_data))
print(f"Total number of records filtered from discharges.csv is : %d" %len(dis_data))

Total number of records filtered from admissions.csv is : 160112
Total number of records filtered from discharges.csv is : 164370


In [62]:
# Apply further filters on columns of interest based on the lookup document for features coded in the csvs
# List the columns from admissions and discharge datasets
adm_data.columns

Index(['CASEID', 'STFIPS (Census state FIPS code)', 'AGE', 'SERVICES', 'SUB1',
       'SUB2', 'SUB3', 'DETCRIM', 'NOPRIOR', 'PSOURCE', 'ARRESTS', 'RACE',
       'ETHNIC', 'SEX', 'EDUC', 'EMPLOY', 'METHUSE', 'PSYPROB', 'PREG', 'VET',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT',
       'DAYWAIT', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2',
       'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ_ATND_SELF_HELP', 'DSMCRIT',
       'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG', 'OPSYNFLG',
       'PCPFLG', 'HALLFLG', 'MTHAMFLG', 'AMPHFLG', 'STIMFLG', 'BENZFLG',
       'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG', 'OTHERFLG',
       'DIVISION', 'REGION', 'ALCDRUG', 'YEAR', 'CBSA10', 'IDU'],
      dtype='object')

In [63]:
dis_data.columns

Index(['DISYR', 'CASEID', 'STFIPS', 'AGE', 'SERVICES', 'SUB1', 'SUB2', 'SUB3',
       'DETCRIM', 'NOPRIOR', 'PSOURCE', 'ARRESTS', 'RACE', 'ETHNIC', 'EDUC',
       'EMPLOY', 'METHUSE', 'PSYPROB', 'PREG', 'VET', 'LIVARAG', 'PRIMINC',
       'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT', 'DAYWAIT', 'ROUTE1', 'FREQ1',
       'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'ROUTE3', 'FREQ3',
       'FRSTUSE3', 'FREQ_ATND_SELF_HELP', 'DSMCRIT', 'SERVICES_D', 'REASON',
       'SUB1_D', 'SUB2_D', 'SUB3_D', 'EMPLOY_D', 'LIVARAG_D', 'DETNLF_D',
       'FREQ1_D', 'FREQ2_D', 'FREQ3_D', 'FREQ_ATND_SELF_HELP_D', 'LOS',
       'ARRESTS_D', 'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG',
       'OPSYNFLG', 'PCPFLG', 'HALLFLG', 'MTHAMFLG', 'AMPHFLG', 'STIMFLG',
       'BENZFLG', 'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG',
       'OTHERFLG', 'NUMSUBS', 'IDU', 'DIVISION', 'REGION', 'ALCDRUG', 'YEAR',
       'CBSA', 'GENDER'],
      dtype='object')

In [64]:
# Choose features of interest for further analysis for this project
adm_study_data = adm_data[['CASEID', 'STFIPS (Census state FIPS code)','AGE', 'SERVICES', 'SUB1', 'SUB2', 
       'DETCRIM', 'NOPRIOR', 'PSOURCE', 'SEX', 'EDUC', 'EMPLOY', 'METHUSE', 'PSYPROB',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT','DAYWAIT', 'ROUTE1', 
       'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2','FREQ_ATND_SELF_HELP', 'DSMCRIT',
       'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'OPSYNFLG','HALLFLG', 'BENZFLG','OTHERFLG',
       'REGION', 'ALCDRUG', 'YEAR', 'IDU']]
# Get the dimensions of the dataset
adm_study_data.shape

(160112, 41)

In [65]:
dis_study_data = dis_data[['CASEID', 'STFIPS', 'AGE', 'SERVICES_D','SUB1_D', 'SUB2_D', 
       'DETCRIM', 'NOPRIOR', 'PSOURCE','GENDER','EDUC','EMPLOY', 'METHUSE', 'PSYPROB',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT', 'DAYWAIT', 'ROUTE1', 
       'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'FREQ_ATND_SELF_HELP', 'DSMCRIT', 
       'REASON', 'EMPLOY_D', 'LIVARAG_D', 'DETNLF_D', 'FREQ1_D', 'FREQ2_D', 
       'FREQ_ATND_SELF_HELP_D', 'LOS','ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'OPSYNFLG', 
       'HALLFLG', 'BENZFLG','OTHERFLG', 'NUMSUBS', 'IDU', 'REGION', 'ALCDRUG', 'DISYR']]
# Get the dimensions of the dataset
dis_study_data.shape

(164370, 50)

In [66]:
# Rename some of the columns to merge dataframes without conflicts
#df1.merge(df2.rename(columns={'b':'a'}),how='outer')
adm_study_data = adm_study_data.rename(columns={'STFIPS (Census state FIPS code)':'STFIPS',
                                                'SEX': 'GENDER'})

In [75]:
# Concat the two datasets
concat_sub_abuse_data = pd.concat([adm_study_data, dis_study_data], axis=1)
concat_sub_abuse_data

,CASEID,STFIPS,AGE,SERVICES,SUB1,SUB2,DETCRIM,NOPRIOR,PSOURCE,GENDER,...,HERFLG,OPSYNFLG,HALLFLG,BENZFLG,OTHERFLG,NUMSUBS,IDU,REGION,ALCDRUG,DISYR
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,2.0,2016.0
5,2.016157e+10,2.0,3.0,7.0,2.0,1.0,-9.0,1.0,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,3.0,0.0,4.0,2.0,2016.0
14,2.016149e+10,2.0,3.0,7.0,2.0,1.0,6.0,3.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2.016130e+10,2.0,4.0,6.0,7.0,2.0,-9.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,3.0,2016.0
20,2.016153e+10,2.0,4.0,7.0,2.0,-9.0,5.0,0.0,7.0,2.0,...,1.0,0.0,0.0,0.0,0.0,3.0,1.0,4.0,2.0,2016.0
27,2.016138e+10,2.0,4.0,7.0,2.0,7.0,-9.0,4.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2.016151e+10,2.0,3.0,7.0,2.0,10.0,4.0,2.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2.016131e+10,2.0,4.0,7.0,2.0,4.0,4.0,1.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
# Concatenate the two datasets on columns.
concat_sub_abuse_data.dropna()

,CASEID,STFIPS,AGE,SERVICES,SUB1,SUB2,DETCRIM,NOPRIOR,PSOURCE,GENDER,...,HERFLG,OPSYNFLG,HALLFLG,BENZFLG,OTHERFLG,NUMSUBS,IDU,REGION,ALCDRUG,DISYR
20,2.016153e+10,2.0,4.0,7.0,2.0,-9.0,5.0,0.0,7.0,2.0,...,1.0,0.0,0.0,0.0,0.0,3.0,1.0,4.0,2.0,2016.0
36,2.016158e+10,2.0,4.0,7.0,2.0,1.0,5.0,0.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,3.0,2016.0
57,2.016136e+10,2.0,3.0,7.0,2.0,1.0,5.0,1.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,2.0,2016.0
103,2.016141e+10,2.0,3.0,6.0,2.0,4.0,6.0,5.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,2016.0
146,2.016161e+10,2.0,4.0,7.0,4.0,2.0,4.0,0.0,7.0,2.0,...,1.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,2.0,2016.0
173,2.016157e+10,2.0,4.0,7.0,5.0,4.0,6.0,1.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,2016.0
231,2.016145e+10,2.0,3.0,7.0,5.0,10.0,-9.0,2.0,2.0,2.0,...,1.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,2.0,2016.0
270,2.016145e+10,2.0,4.0,7.0,2.0,4.0,-9.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,3.0,2016.0
274,2.016154e+10,2.0,3.0,7.0,2.0,4.0,-9.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,2016.0
310,2.016163e+10,2.0,4.0,7.0,2.0,4.0,-9.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,3.0,2016.0


In [79]:
concat_sub_abuse_data.columns

Index(['CASEID', 'STFIPS', 'AGE', 'SERVICES', 'SUB1', 'SUB2', 'DETCRIM',
       'NOPRIOR', 'PSOURCE', 'GENDER', 'EDUC', 'EMPLOY', 'METHUSE', 'PSYPROB',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT',
       'DAYWAIT', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2',
       'FREQ_ATND_SELF_HELP', 'DSMCRIT', 'ALCFLG', 'COKEFLG', 'MARFLG',
       'HERFLG', 'OPSYNFLG', 'HALLFLG', 'BENZFLG', 'OTHERFLG', 'REGION',
       'ALCDRUG', 'YEAR', 'IDU', 'CASEID', 'STFIPS', 'AGE', 'SERVICES_D',
       'SUB1_D', 'SUB2_D', 'DETCRIM', 'NOPRIOR', 'PSOURCE', 'GENDER', 'EDUC',
       'EMPLOY', 'METHUSE', 'PSYPROB', 'LIVARAG', 'PRIMINC', 'HLTHINS',
       'PRIMPAY', 'DETNLF', 'MARSTAT', 'DAYWAIT', 'ROUTE1', 'FREQ1',
       'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'FREQ_ATND_SELF_HELP',
       'DSMCRIT', 'REASON', 'EMPLOY_D', 'LIVARAG_D', 'DETNLF_D', 'FREQ1_D',
       'FREQ2_D', 'FREQ_ATND_SELF_HELP_D', 'LOS', 'ALCFLG', 'COKEFLG',
       'MARFLG', 'HERFLG', 'OPSYNFLG',

In [71]:
# Merge the two datasets based on 'CASEID'
merged_sub_abuse_data = pd.merge(adm_study_data, dis_study_data, on='CASEID', how='outer', suffixes=('_adm', '_dis') )
merged_sub_abuse_data.head()

,CASEID,STFIPS_adm,AGE_adm,SERVICES,SUB1,SUB2,DETCRIM_adm,NOPRIOR_adm,PSOURCE_adm,GENDER_adm,...,HERFLG_dis,OPSYNFLG_dis,HALLFLG_dis,BENZFLG_dis,OTHERFLG_dis,NUMSUBS,IDU_dis,REGION_dis,ALCDRUG_dis,DISYR
0,20161570407,2.0,3.0,7.0,2.0,1.0,-9.0,1.0,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20161487219,2.0,3.0,7.0,2.0,1.0,6.0,3.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20161299121,2.0,4.0,6.0,7.0,2.0,-9.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20161527363,2.0,4.0,7.0,2.0,-9.0,5.0,0.0,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20161383667,2.0,4.0,7.0,2.0,7.0,-9.0,4.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
merged_sub_abuse_data.columns

Index(['CASEID', 'STFIPS', 'AGE', 'SERVICES', 'SUB1', 'SUB2', 'DETCRIM',
       'NOPRIOR', 'PSOURCE', 'GENDER', 'EDUC', 'EMPLOY', 'METHUSE', 'PSYPROB',
       'LIVARAG', 'PRIMINC', 'HLTHINS', 'PRIMPAY', 'DETNLF', 'MARSTAT',
       'DAYWAIT', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'ROUTE2', 'FREQ2', 'FRSTUSE2',
       'FREQ_ATND_SELF_HELP', 'DSMCRIT', 'ALCFLG', 'COKEFLG', 'MARFLG',
       'HERFLG', 'OPSYNFLG', 'HALLFLG', 'BENZFLG', 'OTHERFLG', 'REGION',
       'ALCDRUG', 'YEAR', 'IDU', 'SERVICES_D', 'SUB1_D', 'SUB2_D', 'REASON',
       'EMPLOY_D', 'LIVARAG_D', 'DETNLF_D', 'FREQ1_D', 'FREQ2_D',
       'FREQ_ATND_SELF_HELP_D', 'LOS', 'NUMSUBS', 'DISYR'],
      dtype='object')

In [69]:
# Drop NaNs (null) from the merged dataset
# This is because, we need cases that have a record of admission with a subsequent discharge. Therefore, drop all the records that
# do not match (comes with a NaN) upon a 'full outer join'.
merged_sub_abuse_data.dropna()

,CASEID,STFIPS,AGE,SERVICES,SUB1,SUB2,DETCRIM,NOPRIOR,PSOURCE,GENDER,...,REASON,EMPLOY_D,LIVARAG_D,DETNLF_D,FREQ1_D,FREQ2_D,FREQ_ATND_SELF_HELP_D,LOS,NUMSUBS,DISYR
0,20161044751,1,3,7,4,10,-9,0,6,2,...,2,1,3,-9,1,-9,1,31,2,2016


In [70]:
# Drop all rows that have been coded with -9 for missing or invalid data 
merged_sub_abuse_data = merged_sub_abuse_data[~(merged_sub_abuse_data == '-9.0').any(axis=1)]
merged_sub_abuse_data

,CASEID,STFIPS,AGE,SERVICES,SUB1,SUB2,DETCRIM,NOPRIOR,PSOURCE,GENDER,...,REASON,EMPLOY_D,LIVARAG_D,DETNLF_D,FREQ1_D,FREQ2_D,FREQ_ATND_SELF_HELP_D,LOS,NUMSUBS,DISYR
0,20161044751,1,3,7,4,10,-9,0,6,2,...,2,1,3,-9,1,-9,1,31,2,2016
